In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split

tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")



y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

170498071/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
from sklearn.model_selection import train_test_split
porc_unlabel = 0.99
x_label, x_unlabel, y_label, y_unlabel = train_test_split(x_train, y_train, test_size=porc_unlabel, random_state=10, stratify=y_train)




In [ ]:
y_fake_label = np.zeros(shape=(len(y_unlabel), 10))

print(x_test.shape, y_test.shape)
print(x_label.shape, y_label.shape)
print(x_unlabel.shape, y_fake_label.shape)

(10000, 32, 32, 3) (10000, 10)
(500, 32, 32, 3) (500, 10)
(49500, 32, 32, 3) (49500, 10)


In [ ]:

# uno etiquetados con no etiquetados

# combino etiquetados con los no 
if x_unlabel.shape[0]>0:
  x_train = np.concatenate((x_label, x_unlabel))
  y_train = np.concatenate((y_label, y_fake_label))
  y_real = np.concatenate((y_label, y_unlabel))
else:
  x_train = x_label
  y_train = y_label








In [ ]:
from tensorflow.keras.applications import VGG16


cnn_input = tf.keras.Input(shape=(32, 32, 3))
upsample = tf.keras.layers.UpSampling2D()(cnn_input)
vgg16 = VGG16(include_top=False, weights='imagenet', 
              pooling='avg', input_shape=(64,64,3))(upsample)





model_cnn = keras.models.Model(cnn_input, vgg16)

model_cnn.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 64, 64, 3)        0         
 )                                                               
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def decay(epoch):
  """ This method create the alpha"""
  # returning a very small constant learning rate
  return 0.001 / (1 + 1 * 30)

In [ ]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()


classifier_input = tf.keras.Input(shape=(512,))
### ACÁ AGREGAR MAS CAPAS QUE PUDIERAN SER RELEVANTES, SOLO HAY UNA CAPA DE 32
classifier_dense2 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform')(classifier_input)
classifier_dropout2 = tf.keras.layers.Dropout(0.2)(classifier_dense2)
classifier_dense3 = tf.keras.layers.Dense(len(y_train[0]), activation="softmax")(classifier_dropout2)
classifier = tf.keras.Model(classifier_input, classifier_dense3, name="classifier")

classifier.summary()
cnn_classif_input = tf.keras.Input(shape=(32, 32, 3))
cnn_series = model_cnn(cnn_classif_input)
classif_series = classifier(cnn_series)


final_classif = tf.keras.Model(cnn_classif_input, classif_series, name="CnnClf")


def my_loss(y_truee, y_pred):
    cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    l1 = cce(y_truee, y_pred) 
    mask = tf.reduce_sum(y_truee,axis=1)
    mloss = tf.multiply(mask,l1)
    mloss = tf.reduce_sum(mloss/tf.math.maximum(tf.reduce_sum(mask),1))
    return mloss

final_classif.compile(
loss = my_loss,
optimizer= "adam",
  metrics=['accuracy'])

callback = []
callback += [tf.keras.callbacks.LearningRateScheduler(decay, verbose=1)]
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callback += [es]


final_classif.fit(x=x_train, y=y_train,
  batch_size=128,
  validation_data=(x_test , y_test),
  epochs=2, shuffle=True,
  callbacks=callback,
  verbose=1
  )


Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 512)]             0         
                                                                 
 dense (Dense)               (None, 32)                16416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 16,746
Trainable params: 16,746
Non-trainable params: 0
_________________________________________________________________

Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/2
391/391 [==============================] - 126s 291ms/step - loss: 1.6459 - accuracy: 0.1469 

In [ ]:
from sklearn.cluster import KMeans
from tensorflow.keras.utils import to_categorical
from sklearn.mixture import GaussianMixture

# 5 veces mas la cantidad de etiquetas
k = 5


features = model_cnn.predict(x_train)
features_test =  model_cnn.predict(x_test)

## KMEANS
k_means = KMeans(k*len(y_train[0]),random_state=10).fit(np.concatenate([features, features_test]))
predict2 = k_means.predict(features)  
predict_test =  k_means.predict(features_test) 

## GAUSSIAN MIXTURE
#k_gaussian = GaussianMixture(k*len(y_train[0]),random_state=10).fit(features)
#predict2 = k_gaussian.predict_proba(features)


  
y_kmeans_onehot = to_categorical(predict2, num_classes=k*len(y_train[0])) # pseudo - etiquetas 
y_kmeans_onehot_test = to_categorical(predict_test, num_classes=k*len(y_train[0]))# pseudo - etiquetas  test




313/313 [==============================] - 8s 25ms/step


In [ ]:
y_kmeans_onehot.shape


(50000, 50)